We've covered simple linear regression, but that model is rather limited. It's extremely basic, dealing with only one outcome variable and one input. Most of the time we're interested in a relationship between more than two variables. So what can least squares regression do in that circumstance?

The answer is it can still do quite a lot. Least squares regression can handle almost any functional form you pass into it, it will simply estimate the coefficients.

## Multivariable Least Squares

When a least squares regression has more than one independent variable (or input) it is called multivariable least squares linear regression, multiple linear regression, or just multivariable regression. [Note that this is different from "multivariate" regression, which involves more than one _dependent_ variable.]

This type of model is used when there are multiple variables affecting your outcome. For example, let's think about rental costs. If we wanted to model the monthly rental cost of an apartment there are several factors that matter. We could do something simple, say:

$$ Rent = \alpha + SquareFootage $$

And that might not be a terrible model. It makes sense that as square footage of an apartment goes up so would the cost of rent. However, there's probably more to the story than that.

$$ Rent = \alpha + SquareFootage + Bedrooms + Bathrooms + YearBuilt $$

This would seem like a much more robust model. Now all of these relationships are still linear, meaning each additional bedroom affects rent in the same way. The delta to rent will be equivalent for moving from 1 to 2 bedrooms and from 11 to 12. Again, this is the nature of linear regression. As with the simpler models, here again the linear regression algorithm arrives at estimates for coefficients in front of each variable that minimize the sum of squared errors. We'll cover how that algorithm functions in more detail later.

## Categorical Variables

Not all variables have an obvious linear relationship, however. Categorical variables, for one, pose a unique problem here. A __categorical variable__, much like the outcomes we studied when working with classifiers, is a type or category. This can be something like a boolean {True, False} or state {WA, CA, OR, NV...}. Implying a single linear relationship between all of these values wouldn't make sense. Instead we treat each of them individually, allowing for each value to have its own associated coefficient. So if we wanted to include neighborhood in our example model we could do something like:

$$ Rent = \alpha + SquareFootage + Bedrooms + Bathrooms + YearBuilt + Neighborhood $$

In some languages, such as R, this is functionally sufficient and the algorithm will break neighborhood into categories for the various neighborhoods passed into the model. For Python, however, we have to do a little more work. Because these different categories are not linearly related, SKLearn wants separate binary __indicators__ for each. So for every categorical value we have to create a column that is 0 or 1 indicating if the observation is of that category.

To continue with our rent example, let's say our town was broken into three neighborhoods: Downtown, South, and North. Then our specification would look something like:

$$ Rent = \alpha + SquareFootage + Bedrooms + Bathrooms + YearBuilt + Downtown + South $$

There are a few things to note here. Firstly, one category will always be excluded. That's because it will be incorporated into the intercept (the $\alpha$) and we would otherwise run into collinearity problems, which we'll cover more in the next section. Also, each of these categories would typically be mutually exclusive, which makes sense. If a house is in the South, it shouldn't simultaneously be in the North.

Python offers you two real ways to deal with creating these indicators. The first is to do it manually. If this seems hard or complicated to you THEN YOU SHOULD ABSOLUTELY DO IT! It's good programming practice. However, SKLearn (of course) has a [built in function](http://scikit-learn.org/stable/modules/preprocessing.html) called `OneHotEncoder` that will also do this kind of data preparation.

## Linear doesn't have to mean lines...

Now it is worth noting that the model assumes a linear relationship between the predictors and the outcome, but the predictors themselves can take on more complicated forms. For example, if you specified a functional form of:

$$ y = \alpha + x + x^2 $$

That is perfectly doable in multivariable regression as long as you pass through a dataframe where one column is $x$ and the other column is $x^2$. It is not uncommon to see things like sine or cosine when dealing with a seasonal time series. You can get as complicated as you want, though be careful. Adding a lot of terms can destroy your model by violating the assumptions of OLS. We'll get into those assumptions in the next lesson.

## DRILL: Prepare the Data

[Download the Excel file here](https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/table-8/table-8-state-cuts/table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls) on crime data in New York State in 2013, provided by the FBI: UCR ([Thinkful mirror](https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv)).

Prepare this data to model with multivariable regression (including data cleaning if necessary) according to this specification:

$$ Property crime = \alpha + Population + Population^2 + Murder + Robbery$$

The 'population' variable is already set for you, but you will need to create the last three features.  Robbery and Murder are currently continuous variables.  For this model, please use these variables to create  categorical features where values greater than 0 are coded 1, and values equal to 0 are coded 0.  You'll use this data and model in a later assignment- for now, just write the code you need to get the data ready.  Don't forget basic data cleaning procedures, either!  Do some graphing to see if there are any anomalous cases, and decide how you want to deal with them.


In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format



In [2]:
ny_crime_data = pd.read_csv('ny_offenses_known_to_law_enforcement_new_york_by_city_2013.csv', skiprows=4, skipfooter=3, engine='python')

ny_crime_data


,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Unnamed: 13
0,Adams Village,"1,861",0,0,nan,0,0,0,12,2,10,0,0.000,nan
1,Addison Town and Village,"2,577",3,0,nan,0,0,3,24,3,20,1,0.000,nan
2,Akron Village,"2,846",3,0,nan,0,0,3,16,1,15,0,0.000,nan
3,Albany,"97,956",791,8,nan,30,227,526,"4,090",705,"3,243",142,nan,nan
4,Albion Village,"6,388",23,0,nan,3,4,16,223,53,165,5,nan,nan
5,Alfred Village,"4,089",5,0,nan,0,3,2,46,10,36,0,nan,nan
6,Allegany Village,"1,781",3,0,nan,0,0,3,10,0,10,0,0.000,nan
7,Amherst Town,"118,296",107,1,nan,7,31,68,"2,118",204,"1,882",32,3.000,nan
8,Amityville Village,"9,519",9,0,nan,2,4,3,210,16,188,6,1.000,nan
9,Amsterdam,"18,182",30,0,nan,0,12,18,405,99,291,15,0.000,nan


In [3]:
ny_crime_data.dropna(axis=1, how='all', inplace=True)
ny_crime_data.dtypes

City                                       object
Population                                 object
Violent\ncrime                             object
Murder and\nnonnegligent\nmanslaughter      int64
Rape\n(legacy\ndefinition)2                object
Robbery                                    object
Aggravated\nassault                        object
Property\ncrime                            object
Burglary                                   object
Larceny-\ntheft                            object
Motor\nvehicle\ntheft                      object
Arson3                                    float64
dtype: object

In [4]:
from sklearn.impute import SimpleImputer


In [5]:
ny_crime_data2 = pd.DataFrame()
ny_crime_data2['City'] = ny_crime_data['City']
ny_crime_data2['Population'] = ny_crime_data.Population.str.replace(',' , '').astype(int)
ny_crime_data2['Population2'] = ny_crime_data2['Population']**2
ny_crime_data2['Robbery'] = ny_crime_data.Robbery.str.replace(',' , '').astype(int)
ny_crime_data2['Robbery_bi'] = ny_crime_data2['Robbery'].apply(lambda x: 1 if x > 0 else 0)
ny_crime_data2['Murder'] = ny_crime_data['Murder and\nnonnegligent\nmanslaughter']
ny_crime_data2['Murder_bi'] = ny_crime_data2['Murder'].apply(lambda x: 1 if x > 0 else 0)
ny_crime_data2['Property Crime'] = 
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
ny_crime_data2.head()


,City,Population,Population2,Robbery,Robbery_bi,Murder,Murder_bi
0,Adams Village,1861,3463321,0,0,0,0
1,Addison Town and Village,2577,6640929,0,0,0,0
2,Akron Village,2846,8099716,0,0,0,0
3,Albany,97956,1005443344,227,1,8,1
4,Albion Village,6388,40806544,4,1,0,0


In [6]:
ny_crime_data2.dtypes

City           object
Population      int32
Population2     int32
Robbery         int32
Robbery_bi      int64
Murder          int64
Murder_bi       int64
dtype: object

In [7]:
#col_ls=ny_crime_data.columns.tolist()
#new_df=ny_crime_data[['Arson3']]
#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
#print((imp.fit_transform(new_df)))

In [14]:
# Instantiate our model.
regr = linear_model.LinearRegression()
x = ny_crime_data2['Robbery_bi']
y = ny_crime_data2['Murder_bi']
# Fit our model to our data.
regr.fit(x, y)

# Display the attributes we calculated.
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)

# Plot outputs
plt.scatter(x, y, color='black')
plt.plot(x, regr.predict(x), color='blue', linewidth=3)
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0
 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0
 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 1
 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1
 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0
 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0
 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 0 0 0 0
 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0
 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.